In [1]:
import numpy as np
import random
import pandas 
import pickle
import os
import time
from preprocessing_dailydialogue import *

In [19]:
with open('../data/ecm/ecm_data1.pickle','rb') as f:
    pairs = pickle.load(f)
    pairs_emotion = pickle.load(f)
    
with open('../data/ecm/ecm_data2.pickle','rb') as f:
    pairs2 = pickle.load(f)
    pairs_emotion2 = pickle.load(f)

In [20]:
pairs = pairs + pairs2
pairs_emotion = pairs_emotion + pairs_emotion2

In [4]:
index = range(len(pairs))
index_train = random.sample(index,303421)
index = set(index) - set(index_train)
index_test = random.sample(index,5000)
index_val = list(set(index) - set(index_test))

In [5]:
train =[]
train_emotion =[]
for each in index_train:
    train.append(pairs[each])
    train_emotion.append(pairs_emotion[each])
    
test =[]
test_emotion =[]
for each in index_test:
    test.append(pairs[each])
    test_emotion.append(pairs_emotion[each])
    
val =[]
val_emotion =[]
for each in index_val:
    val.append(pairs[each])
    val_emotion.append(pairs_emotion[each])

In [28]:
with open('train.pickle','wb') as f:
    pickle.dump(train,f)
    pickle.dump(train_emotion,f)

In [29]:
with open('test.pickle','wb') as f:
    pickle.dump(test,f)
    pickle.dump(test_emotion,f)

In [30]:
with open('val.pickle','wb') as f:
    pickle.dump(val,f)
    pickle.dump(val_emotion,f)

In [23]:
with open('val.pickle','rb') as f:
    a = pickle.load(f)
    b = pickle.load(f)

# Preprocessing

In [2]:
# load from begining .
# Default word tokens
#
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token
MAX_LENGTH = 10  # Maximum sentence length to consider
MIN_COUNT = 3    # Minimum word count threshold for trimming
save_dir = os.path.join("data", "save")
emo_dict = { 0: 'neutral', 1: 'joy', 2: 'anger', 
            3: 'sadness',4:'fear'}
emo2idx = {value:key for key,value in emo_dict.items()}

with open('../data/ecm/train.pickle','rb') as f:
    pairs = pickle.load(f)
    pairs_emotion = pickle.load(f)
def process(pairs,pairs_emotion,voc = None,emo2idx = emo2idx):   
    pairs_emotion_index = []
    for each in pairs_emotion:
        pairs_emotion_index.append([emo2idx[each[0]],emo2idx[each[1]]])
    pairs_emotion = pairs_emotion_index
    del pairs_emotion_index
    if voc is None:
        voc = Voc(name='train',max_length=MAX_LENGTH,min_count=MIN_COUNT)
        print(len(pairs))
        pairs,pairs_emotion = filterPairs(pairs,pairs_emotion,voc.max_length)
        print(len(pairs),len(pairs_emotion))
        for each in pairs:
            voc.addSentence(each[0])
            voc.addSentence(each[1])
        print(len(pairs))
    else:
        pairs,pairs_emotion = filterPairs(pairs,pairs_emotion,voc.max_length)
    pairs,pairs_emotion = trimRareWords(pairs=pairs,pairs_emotion=pairs_emotion,voc=voc,min_count=voc.min_count)
    print(len(pairs),len(pairs_emotion))
    
    return pairs,pairs_emotion,voc



In [3]:
pairs,pairs_emotion,voc = process(pairs,pairs_emotion)

303421
81873 81873
81873
keep_words 8848 / 19419 = 0.4556
Trimmed from 81873 pairs to 70044, 0.8555 of total
70044 70044


In [4]:
with open('../data/ecm/test.pickle','rb') as f:
    pairs_t = pickle.load(f)
    pairs_emotion_t = pickle.load(f)

In [5]:
len(pairs_t),len(pairs_emotion_t)

(5000, 5000)

In [6]:
pairs_t,pairs_emotion_t,voc_t = process(pairs_t,pairs_emotion_t,voc)

Trimmed from 1345 pairs to 1063, 0.7903 of total
1063 1063


In [7]:
print(pairs[:5],pairs_emotion[:5])
print(pairs_t[:5],pairs_emotion_t[:5])

[['yes there was .', 'can you tell me what happened ?'], ['damone . it s mark .', 'mark . what happened to your date ?'], ['do i know you from somewhere ?', 'you may have known my father william starks .'], ['look .', 'you re almost one of us now michael .'], ['you should have gone at the restaurant .', 'i didn t have to pee then .']] [[2, 2], [2, 0], [4, 4], [4, 2], [4, 3]]
[['i m running away .', 'you think that s wise ?'], ['can we see ?', 'you gonna buy ?'], ['i read the hospital report on your client .', '. . . deborah ann kaye . . .'], ['why the fuck not ?', 'just don t .'], ['what is it ? !', 'i don t know . wave to them !']] [[4, 0], [2, 4], [4, 2], [2, 4], [0, 2]]


In [8]:
len(pairs_t),len(pairs_emotion_t)

(1063, 1063)

In [9]:
for each in pairs_t:
    if len(each[0].split(' ')) >= voc.max_length:
        print(each)

In [10]:
with open('processed_train.pickle','wb') as f:
    pickle.dump(pairs,f)
    pickle.dump(pairs_emotion,f)
    pickle.dump(voc,f)
    
with open('processed_test.pickle','wb') as f:
    pickle.dump(pairs_t,f)
    pickle.dump(pairs_emotion_t,f)

In [10]:
len(pairs),len(pairs_emotion)

(214361, 214361)